In [1]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# Load data

1. Get html content using `requests.get`
2. Parse the html using `BeautifulSoup`
3. Load the parsed table into pandas dataframe using `pd.read_html`
4. Drop `NaN` values in column Borough
5. Replace `NaN` values in column Neighbour with the value from Borough (ZERO case found for the dadtaset)
6. Merge two neighbourhoods with the same Postal Code (ZERO case found for the dataset)
7. Reorder the dataset

In [2]:
# auxiliary function to merge duplicated postal code that does not exist
def merge_duplicate(df):
    """
    Find all the rows in [df] with same 'Postal Code' value
        then join their 'Neighbourhood' values, inplace

    Example:
        >>> test = pd.DataFrame({ \
                'Postal Code': ('M1A', 'M1A', 'M2A', 'M3A', 'M1A', 'M2A'), \
                'Neighbourhood': ('A', 'B', 'C', 'D', 'E', 'F'), \
            })
        >>> merge_duplicate(test)
        >>> test.loc[test['Postal Code'] == 'M1A']['Neighbourhood'].values[0]
        'A, B, E'
        >>> test.loc[test['Postal Code'] == 'M2A']['Neighbourhood'].values[0]
        'C, F'
    """
    counts = df['Postal Code'].value_counts()
    duplicate = set([code for code, c in zip(counts.index, counts) if c > 1])
    
    for code in duplicate:
        indices = np.where(df['Postal Code'] == code)[0]
        neighbour_names = df['Neighbourhood'][indices]
        df['Neighbourhood'][indices[0]] = ', '.join(neighbour_names)
        df.drop(indices[1:], inplace=True)


# step 1
url_to_parse = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url_to_parse).text

# step 2
parsed = BeautifulSoup(html)
table_html = str(parsed.table)

# step 3
df = pd.read_html(table_html)[0]

# step 4
df.replace({'Borough' : {'Not assigned' : np.nan}}, inplace=True)
df.dropna(inplace=True)

# step 5
replaced = np.where(
    df['Neighbourhood'] == 'not assigned',  # the condition
    df['Borough'],  # choose value from array if condition is True
    df['Neighbourhood'],  # choose value from array if condition is False
)
df['Neighbourhood'] = replaced

# step 6
merge_duplicate(df)

# step 7
df.reset_index(inplace=True, drop=True)
df.shape

(103, 3)

# Inspect the data

In [3]:
df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
